In [2]:
# SPAM Detection- SLM
# A prediction of 1 typically represents a legitimate (non-spam) email
# A prediction of 0 typically represents a spam email
# Code snippet 1. Run the Data prepration, fine tuning, inference
# Code snippet 2. Edge Deployment AWS

In [3]:
!pip install datasets==2.14.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00


In [4]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

# Prepare data
data = {
    'text': [
        "You've won a gift card! Click here.",
        "I need assistance with my order.",
        "Congratulations! You are our lucky winner!",
        "Regarding your recent purchase...",
        "Claim your free vacation!",
        "Thank you for your business.",
        "Urgent: Your account requires verification",
        "Meeting scheduled for tomorrow.",
        "Limited-time offer! Don't miss out!",
        "Invoice attached for your review."
    ],
    'label': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1: Spam, 0: Not Spam
}

df = pd.DataFrame(data)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")

# Inference function
def inference(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Example inference
test_texts = ["You've won a prize!", "Please review the attached document."]
predictions = inference(test_texts)
print("Predictions:", predictions)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anvcse2007 (anvcse2007-university-of-illinois-urbana-champaign) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss


Predictions: tensor([1, 1])


In [ ]:
#ata preparation, fine-tuning, inference, and steps for edge deployment using AWS services.

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sagemaker.huggingface import HuggingFace
from sagemaker.neo import NeoCompilationJob
from sagemaker.edge import EdgePackagingJob

# Step 1: Prepare dummy data
data = {
    'text': [
        "You've won a gift card! Click here.",
        "I need assistance with my order.",
        "Congratulations! You are our lucky winner!",
        "Regarding your recent purchase...",
        "Claim your free vacation!",
        "Thank you for your business.",
        "Urgent: Your account requires verification",
        "Meeting scheduled for tomorrow.",
        "Limited-time offer! Don't miss out!",
        "Invoice attached for your review."
    ],
    'label': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1: Spam, 0: Not Spam
}

df = pd.DataFrame(data)

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save to CSV for SageMaker
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

# Upload to S3 (you need to have AWS CLI configured)
!aws s3 cp train.csv s3://your-bucket/spam-detection/train.csv
!aws s3 cp test.csv s3://your-bucket/spam-detection/test.csv

# Step 2: Fine-tuning (this would typically be in a separate script)
def model_fn(model_dir):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained(model_dir)
    return model, tokenizer

def train():
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

    train_dataset = Dataset.from_pandas(pd.read_csv('train.csv'))
    test_dataset = Dataset.from_pandas(pd.read_csv('test.csv'))

    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True)

    tokenized_train = train_dataset.map(tokenize_function, batched=True)
    tokenized_test = test_dataset.map(tokenize_function, batched=True)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test
    )

    trainer.train()
    trainer.save_model('./model')

# Step 3: SageMaker Training Job
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    instance_type='ml.g5.2xlarge',
    instance_count=1,
    transformers_version='4.26.0',
    pytorch_version='1.13.1',
    py_version='py39',
    role='YourSageMakerRole'
)

huggingface_estimator.fit({
    'train': 's3://your-bucket/spam-detection/train.csv',
    'test': 's3://your-bucket/spam-detection/test.csv'
})

# Step 4: Inference
def inference(model, tokenizer, texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Load the fine-tuned model
model_path = 's3://your-bucket/spam-detection/model'
model, tokenizer = model_fn(model_path)

# Example inference
texts = ["You've won a gift card! Click here.", "I need assistance with my order."]
predictions = inference(model, tokenizer, texts)
print(predictions)

# Step 5: Deployment on Edge

# Compile the model using SageMaker Neo
compilation_job = NeoCompilationJob.compile(
    input_model_s3_uri='s3://your-bucket/spam-detection/model',
    target_instance_family='ml_c5',
    output_model_s3_uri='s3://your-bucket/spam-detection/compiled-model',
    role='YourSageMakerRole',
    framework='PYTORCH',
    framework_version='1.8'
)
compilation_job.wait()

# Package the model for edge deployment
packaging_job = EdgePackagingJob.create(
    role='YourSageMakerRole',
    model_name='spam-detection-model',
    model_version='1.0',
    output_package_directory='s3://your-bucket/spam-detection/edge-package',
    model_artifact='s3://your-bucket/spam-detection/compiled-model',
    target_platform={
        'os': 'linux',
        'arch': 'x86_64'
    }
)
packaging_job.wait()

print("Model packaged for edge deployment")
